<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_6_Configuration_Management_Customizing_Your_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 In previous tutorials, you've built intelligent agents with communication, memory, rules, and monitoring. Now we're adding flexible configuration management so you can easily customize agent behavior, deploy across environments, and manage complex multi-agent systems without hardcoding values.

 What you'll build:

• ConfigManager system for centralized configuration

• Environment-specific configuration loading (dev, staging, prod)

• Dynamic configuration updates without restarts

• Configuration validation and schema enforcement

• Secure configuration with environment variable integration

• Configuration templates and inheritance patterns

Why this matters:

Production AI systems need flexible configuration to adapt to different
environments, scale with changing requirements, and allow non-technical
users to customize behavior. Good configuration management is essential
for maintainable, deployable AI applications.

By the end, you'll understand:

• How to design flexible configuration systems

• Environment-specific deployment patterns

• Configuration validation and error handling

• Security best practices for sensitive configuration

• Dynamic reconfiguration without downtime

In [1]:
print("Tutorial 6: Configuration Management - Customizing Your Agents")
print("=" * 62)
print()
print("Building flexible configuration systems for production AI deployment...")
print()

Tutorial 6: Configuration Management - Customizing Your Agents

Building flexible configuration systems for production AI deployment...



In [4]:
# Essential imports
import uuid
import time
import threading
import queue
import json
import os
import copy
from abc import ABC, abstractmethod
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Optional, Set, Callable, Union, Tuple
from enum import Enum
from collections import defaultdict
from pathlib import Path

# Try to import yaml, create a fallback if not available
try:
    import yaml
except ImportError:
    print("⚠️  PyYAML not available. YAML config files will not be supported.")
    yaml = None

In [5]:
# Import our foundation from previous tutorials
AgentID = str
MessageID = str

class MessagePriority(Enum):
    LOW = 1
    NORMAL = 2
    HIGH = 3
    URGENT = 4

@dataclass
class Message:
    id: MessageID
    sender: AgentID
    recipients: List[AgentID]
    content: Any
    priority: MessagePriority = MessagePriority.NORMAL
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: float = field(default_factory=time.time)

    @classmethod
    def create(cls, sender: AgentID, recipients: List[AgentID], content: Any,
               priority: MessagePriority = MessagePriority.NORMAL) -> 'Message':
        return cls(
            id=str(uuid.uuid4()),
            sender=sender,
            recipients=recipients,
            content=content,
            priority=priority
        )

class ConfigurationError(Exception):
    """Custom exception for configuration-related errors"""
    pass

class ConfigurationType(Enum):
    """Types of configuration data for validation and handling"""
    STRING = "string"
    INTEGER = "integer"
    FLOAT = "float"
    BOOLEAN = "boolean"
    LIST = "list"
    DICT = "dict"
    SECRET = "secret"  # For sensitive data like API keys

@dataclass
class ConfigField:
    """
    Definition of a configuration field with validation rules

    This defines what a configuration field should look like,
    including its type, default value, and validation rules.
    """
    name: str                                    # Field name
    config_type: ConfigurationType              # Data type
    default: Any = None                         # Default value
    required: bool = False                      # Is this field required
    description: str = ""                       # Human-readable description
    min_value: Optional[Union[int, float]] = None  # Minimum value (for numbers)
    max_value: Optional[Union[int, float]] = None  # Maximum value (for numbers)
    allowed_values: Optional[List[Any]] = None  # Allowed values (enum-like)
    validation_regex: Optional[str] = None      # Regex pattern for strings
    environment_variable: Optional[str] = None  # Environment variable name

    def validate(self, value: Any) -> Tuple[bool, str]:
        """
        Validate a value against this field's rules

        Returns (is_valid, error_message)
        """
        if value is None:
            if self.required:
                return False, f"Required field '{self.name}' is missing"
            return True, ""

        # Type validation
        if self.config_type == ConfigurationType.STRING:
            if not isinstance(value, str):
                return False, f"Field '{self.name}' must be a string, got {type(value).__name__}"
        elif self.config_type == ConfigurationType.INTEGER:
            if not isinstance(value, int):
                return False, f"Field '{self.name}' must be an integer, got {type(value).__name__}"
        elif self.config_type == ConfigurationType.FLOAT:
            if not isinstance(value, (int, float)):
                return False, f"Field '{self.name}' must be a number, got {type(value).__name__}"
            value = float(value)
        elif self.config_type == ConfigurationType.BOOLEAN:
            if not isinstance(value, bool):
                return False, f"Field '{self.name}' must be a boolean, got {type(value).__name__}"
        elif self.config_type == ConfigurationType.LIST:
            if not isinstance(value, list):
                return False, f"Field '{self.name}' must be a list, got {type(value).__name__}"
        elif self.config_type == ConfigurationType.DICT:
            if not isinstance(value, dict):
                return False, f"Field '{self.name}' must be a dict, got {type(value).__name__}"

        # Range validation for numbers
        if self.config_type in [ConfigurationType.INTEGER, ConfigurationType.FLOAT]:
            if self.min_value is not None and value < self.min_value:
                return False, f"Field '{self.name}' must be >= {self.min_value}, got {value}"
            if self.max_value is not None and value > self.max_value:
                return False, f"Field '{self.name}' must be <= {self.max_value}, got {value}"

        # Allowed values validation
        if self.allowed_values is not None:
            if value not in self.allowed_values:
                return False, f"Field '{self.name}' must be one of {self.allowed_values}, got {value}"

        # Regex validation for strings
        if self.validation_regex is not None and self.config_type == ConfigurationType.STRING:
            import re
            if not re.match(self.validation_regex, value):
                return False, f"Field '{self.name}' does not match required pattern"

        return True, ""

class ConfigSchema:
    """
    Schema definition for configuration validation

    This defines the structure and validation rules for a complete
    configuration, ensuring that all required fields are present
    and all values are valid.
    """

    def __init__(self, name: str):
        self.name = name
        self.fields: Dict[str, ConfigField] = {}
        self.sections: Dict[str, 'ConfigSchema'] = {}  # Nested schemas

    def add_field(self, field: ConfigField):
        """Add a field definition to this schema"""
        self.fields[field.name] = field
        return self  # For chaining

    def add_section(self, section_name: str, schema: 'ConfigSchema'):
        """Add a nested section to this schema"""
        self.sections[section_name] = schema
        return self

    def validate(self, config: Dict[str, Any]) -> Tuple[bool, List[str]]:
        """
        Validate a configuration against this schema

        Returns (is_valid, list_of_errors)
        """
        errors = []

        # Validate fields
        for field_name, field_def in self.fields.items():
            value = config.get(field_name)
            is_valid, error_msg = field_def.validate(value)
            if not is_valid:
                errors.append(f"[{self.name}] {error_msg}")

        # Validate nested sections
        for section_name, section_schema in self.sections.items():
            section_config = config.get(section_name, {})
            if not isinstance(section_config, dict):
                errors.append(f"[{self.name}] Section '{section_name}' must be a dictionary")
                continue

            section_valid, section_errors = section_schema.validate(section_config)
            errors.extend(section_errors)

        return len(errors) == 0, errors

    def get_defaults(self) -> Dict[str, Any]:
        """Get default values for all fields in this schema"""
        defaults = {}

        # Add field defaults
        for field_name, field_def in self.fields.items():
            if field_def.default is not None:
                defaults[field_name] = field_def.default

        # Add section defaults
        for section_name, section_schema in self.sections.items():
            section_defaults = section_schema.get_defaults()
            if section_defaults:
                defaults[section_name] = section_defaults

        return defaults

class ConfigManager:
    """
    Centralized configuration management system

    ConfigManager handles loading, validation, merging, and updating
    of configuration from multiple sources including files, environment
    variables, and runtime updates.
    """

    def __init__(self, schema: ConfigSchema = None):
        self.schema = schema
        self.config: Dict[str, Any] = {}
        self.environment = "development"  # Current environment
        self.config_sources: List[str] = []  # Track where config came from
        self.config_lock = threading.RLock()  # For thread-safe updates
        self.change_listeners: List[Callable] = []  # Callbacks for config changes
        self.last_loaded = None  # When was config last loaded

        print(f"🔧 ConfigManager initialized")
        if schema:
            print(f"   Schema: {schema.name}")

    def set_environment(self, environment: str):
        """Set the current environment (dev, staging, prod, etc.)"""
        self.environment = environment
        print(f"🌍 Environment set to: {environment}")

    def load_from_dict(self, config_dict: Dict[str, Any], source: str = "dict"):
        """Load configuration from a dictionary"""
        with self.config_lock:
            self.config.update(config_dict)
            if source not in self.config_sources:
                self.config_sources.append(source)
            self.last_loaded = time.time()

            print(f"📁 Loaded configuration from {source}")
            self._notify_listeners()

    def load_from_file(self, file_path: str):
        """Load configuration from a YAML or JSON file"""
        path = Path(file_path)

        if not path.exists():
            raise ConfigurationError(f"Configuration file not found: {file_path}")

        try:
            with open(path, 'r') as f:
                if path.suffix.lower() in ['.yaml', '.yml']:
                    if yaml is None:
                        raise ConfigurationError("PyYAML not available for YAML file support")
                    config_data = yaml.safe_load(f) or {}
                elif path.suffix.lower() == '.json':
                    config_data = json.load(f)
                else:
                    raise ConfigurationError(f"Unsupported file format: {path.suffix}")

            self.load_from_dict(config_data, f"file:{file_path}")
            print(f"📄 Loaded configuration from file: {file_path}")

        except Exception as e:
            raise ConfigurationError(f"Error loading configuration file {file_path}: {e}")

    def load_from_environment(self):
        """Load configuration from environment variables"""
        if not self.schema:
            print("⚠️  No schema defined, skipping environment variable loading")
            return

        env_config = {}
        loaded_vars = []

        def extract_env_vars(schema: ConfigSchema, config_dict: Dict[str, Any], prefix: str = ""):
            for field_name, field_def in schema.fields.items():
                env_var = field_def.environment_variable
                if env_var and env_var in os.environ:
                    value = os.environ[env_var]

                    # Convert string environment variable to proper type
                    try:
                        if field_def.config_type == ConfigurationType.INTEGER:
                            value = int(value)
                        elif field_def.config_type == ConfigurationType.FLOAT:
                            value = float(value)
                        elif field_def.config_type == ConfigurationType.BOOLEAN:
                            value = value.lower() in ('true', '1', 'yes', 'on')
                        elif field_def.config_type == ConfigurationType.LIST:
                            value = [item.strip() for item in value.split(',')]
                        elif field_def.config_type == ConfigurationType.DICT:
                            value = json.loads(value)
                        # STRING and SECRET types stay as strings

                        config_dict[field_name] = value
                        loaded_vars.append(env_var)

                    except Exception as e:
                        print(f"⚠️  Error parsing environment variable {env_var}: {e}")

            # Process nested sections
            for section_name, section_schema in schema.sections.items():
                section_config = config_dict.setdefault(section_name, {})
                extract_env_vars(section_schema, section_config, f"{prefix}{section_name}_")

        extract_env_vars(self.schema, env_config)

        if env_config:
            self.load_from_dict(env_config, "environment")
            print(f"🌍 Loaded {len(loaded_vars)} environment variables: {loaded_vars}")
        else:
            print("🌍 No environment variables found for configuration")

    def load_environment_specific(self, base_config_path: str):
        """
        Load environment-specific configuration

        Loads base config first, then overlays environment-specific config.
        For example: config.yaml + config.production.yaml
        """
        base_path = Path(base_config_path)

        # Load base configuration
        if base_path.exists():
            self.load_from_file(str(base_path))
        else:
            print(f"⚠️  Base config file not found: {base_config_path}")

        # Load environment-specific overlay
        env_path = base_path.parent / f"{base_path.stem}.{self.environment}{base_path.suffix}"
        if env_path.exists():
            self.load_from_file(str(env_path))
            print(f"🎯 Applied {self.environment} environment overrides")
        else:
            print(f"ℹ️  No environment-specific config found: {env_path}")

    def apply_defaults(self):
        """Apply default values from schema for missing fields"""
        if not self.schema:
            return

        defaults = self.schema.get_defaults()
        if defaults:
            # Merge defaults with existing config (existing values take precedence)
            merged_config = self._deep_merge(defaults, self.config)
            with self.config_lock:
                self.config = merged_config
            print(f"🔧 Applied default values for missing configuration fields")

    def validate(self) -> Tuple[bool, List[str]]:
        """Validate current configuration against schema"""
        if not self.schema:
            return True, []

        with self.config_lock:
            return self.schema.validate(self.config)

    def get(self, key: str, default: Any = None) -> Any:
        """
        Get a configuration value using dot notation

        Examples: get('database.host'), get('agents.alice.personality')
        """
        with self.config_lock:
            keys = key.split('.')
            value = self.config

            for k in keys:
                if isinstance(value, dict) and k in value:
                    value = value[k]
                else:
                    return default

            return value

    def set(self, key: str, value: Any):
        """
        Set a configuration value using dot notation

        This allows runtime configuration updates.
        """
        with self.config_lock:
            keys = key.split('.')
            config_ref = self.config

            # Navigate to the parent of the target key
            for k in keys[:-1]:
                if k not in config_ref:
                    config_ref[k] = {}
                config_ref = config_ref[k]

            # Set the value
            old_value = config_ref.get(keys[-1])
            config_ref[keys[-1]] = value

            print(f"🔧 Configuration updated: {key} = {value} (was: {old_value})")
            self._notify_listeners()

    def get_all(self) -> Dict[str, Any]:
        """Get a copy of the entire configuration"""
        with self.config_lock:
            return copy.deepcopy(self.config)

    def add_change_listener(self, listener: Callable[['ConfigManager'], None]):
        """Add a callback that's called when configuration changes"""
        self.change_listeners.append(listener)
        print(f"🔔 Added configuration change listener")

    def reload(self):
        """Reload configuration from all sources"""
        print("🔄 Reloading configuration...")

        # Clear current config but remember sources
        sources = self.config_sources.copy()
        with self.config_lock:
            self.config.clear()
            self.config_sources.clear()

        # Reload from each source
        for source in sources:
            if source.startswith("file:"):
                file_path = source[5:]  # Remove "file:" prefix
                try:
                    self.load_from_file(file_path)
                except Exception as e:
                    print(f"⚠️  Error reloading {source}: {e}")
            elif source == "environment":
                self.load_from_environment()

        self.apply_defaults()
        print("✅ Configuration reloaded")

    def _deep_merge(self, base: Dict, overlay: Dict) -> Dict:
        """Deep merge two dictionaries, with overlay taking precedence"""
        result = base.copy()

        for key, value in overlay.items():
            if key in result and isinstance(result[key], dict) and isinstance(value, dict):
                result[key] = self._deep_merge(result[key], value)
            else:
                result[key] = value

        return result

    def _notify_listeners(self):
        """Notify all registered listeners of configuration changes"""
        for listener in self.change_listeners:
            try:
                listener(self)
            except Exception as e:
                print(f"⚠️  Error in configuration change listener: {e}")

    def get_status(self) -> Dict[str, Any]:
        """Get status information about the configuration system"""
        with self.config_lock:
            return {
                'environment': self.environment,
                'sources': self.config_sources,
                'last_loaded': self.last_loaded,
                'config_keys': list(self.config.keys()),
                'has_schema': self.schema is not None,
                'listeners': len(self.change_listeners)
            }

class ConfigurableAgent:
    """
    Base class for agents that can be configured via ConfigManager

    This demonstrates how to build agents that respond to configuration
    changes and can be customized without code changes.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 config_manager: ConfigManager = None):
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or self.__class__.__name__
        self.config_manager = config_manager

        # Message processing
        self._message_queue = queue.Queue()
        self._stop_event = threading.Event()
        self._processing_thread = None
        self._running = False

        # Configuration-driven properties
        self.personality = "balanced"
        self.max_response_length = 200
        self.processing_delay = 0.01
        self.debug_mode = False

        print(f"🤖 Initialized ConfigurableAgent: {self.name}")

        # Load initial configuration
        if self.config_manager:
            self._load_configuration()
            self.config_manager.add_change_listener(self._on_config_change)

    def _load_configuration(self):
        """Load configuration values from ConfigManager"""
        if not self.config_manager:
            return

        # Load agent-specific configuration
        agent_config_key = f"agents.{self.name.lower()}"

        self.personality = self.config_manager.get(f"{agent_config_key}.personality", self.personality)
        self.max_response_length = self.config_manager.get(f"{agent_config_key}.max_response_length", self.max_response_length)
        self.processing_delay = self.config_manager.get(f"{agent_config_key}.processing_delay", self.processing_delay)
        self.debug_mode = self.config_manager.get(f"{agent_config_key}.debug_mode", self.debug_mode)

        # Load global defaults
        self.personality = self.config_manager.get("agents.default.personality", self.personality)
        self.max_response_length = self.config_manager.get("agents.default.max_response_length", self.max_response_length)
        self.processing_delay = self.config_manager.get("agents.default.processing_delay", self.processing_delay)
        self.debug_mode = self.config_manager.get("global.debug_mode", self.debug_mode)

        if self.debug_mode:
            print(f"🔧 {self.name} configuration loaded:")
            print(f"   Personality: {self.personality}")
            print(f"   Max response length: {self.max_response_length}")
            print(f"   Processing delay: {self.processing_delay}s")
            print(f"   Debug mode: {self.debug_mode}")

    def _on_config_change(self, config_manager: ConfigManager):
        """Callback when configuration changes"""
        if self.debug_mode:
            print(f"🔔 {self.name} received configuration change notification")

        old_personality = self.personality
        self._load_configuration()

        if old_personality != self.personality:
            print(f"🎭 {self.name} personality changed: {old_personality} → {self.personality}")

    def start(self):
        """Start the agent"""
        if self._running:
            print(f"⚠️  Agent {self.name} is already running")
            return

        self._stop_event.clear()
        self._processing_thread = threading.Thread(
            target=self._processing_loop,
            daemon=True,
            name=f"ConfigurableAgent-{self.name}"
        )
        self._processing_thread.start()
        self._running = True

        print(f"▶️  ConfigurableAgent {self.name} started")

    def stop(self):
        """Stop the agent"""
        if not self._running:
            return

        self._stop_event.set()

        if self._processing_thread:
            self._processing_thread.join(timeout=2.0)

        self._running = False
        print(f"⏹️  ConfigurableAgent {self.name} stopped")

    def receive_message(self, message: Message):
        """Receive a message for processing"""
        self._message_queue.put(message)

        if self.debug_mode:
            print(f"📬 {self.name} received: {str(message.content)[:50]}{'...' if len(str(message.content)) > 50 else ''}")

    def process_message(self, message: Message):
        """Process a message using current configuration"""
        content = str(message.content).lower()

        if self.debug_mode:
            print(f"🎯 {self.name} ({self.personality}) processing: {message.content}")

        # Generate response based on personality and configuration
        response = self._generate_response(content)

        # Truncate response if needed
        if len(response) > self.max_response_length:
            response = response[:self.max_response_length-3] + "..."

        # Apply configured processing delay
        if self.processing_delay > 0:
            time.sleep(self.processing_delay)

        # Send response
        response_msg = Message.create(
            sender=self.id,
            recipients=[message.sender],
            content=response
        )

        print(f"📤 {self.name} → {message.sender[:8]}...: {response}")

        return response_msg

    def _generate_response(self, content: str) -> str:
        """Generate personality-based responses using configuration"""

        # Personality-based responses that can be configured
        personality_responses = {
            "friendly": {
                "greeting": "Hello there! It's wonderful to meet you! 😊",
                "question": "That's such an interesting question! I love discussing this!",
                "farewell": "Goodbye! It was so nice talking with you! 👋",
                "default": "That's really cool! Thanks for sharing!"
            },
            "professional": {
                "greeting": "Good day. I'm pleased to assist you.",
                "question": "I'll be happy to address your inquiry.",
                "farewell": "Thank you for your time. Have a productive day.",
                "default": "I understand. Please let me know if you need clarification."
            },
            "casual": {
                "greeting": "Hey! What's up?",
                "question": "Good question! Let me think about that...",
                "farewell": "Catch you later!",
                "default": "Cool, got it!"
            },
            "analytical": {
                "greeting": "Hello. I'm ready to analyze any topics you'd like to discuss.",
                "question": "Let me examine this systematically. Based on available data...",
                "farewell": "Goodbye. This conversation provided valuable data points.",
                "default": "Interesting. Let me process this information."
            }
        }

        # Get responses for current personality, fall back to friendly
        responses = personality_responses.get(self.personality, personality_responses["friendly"])

        # Determine response type
        if any(word in content for word in ["hello", "hi", "hey"]):
            return responses["greeting"]
        elif "?" in content or any(word in content for word in ["what", "how", "why"]):
            return responses["question"]
        elif any(word in content for word in ["bye", "goodbye", "farewell"]):
            return responses["farewell"]
        else:
            return responses["default"]

    def _processing_loop(self):
        """Main message processing loop"""
        if self.debug_mode:
            print(f"🔄 {self.name} processing loop started")

        while not self._stop_event.is_set():
            try:
                message = self._message_queue.get(timeout=0.1)

                start_time = time.time()
                self.process_message(message)
                processing_time = time.time() - start_time

                self._message_queue.task_done()

                if self.debug_mode:
                    print(f"✅ {self.name} processed message in {processing_time:.3f}s")

            except queue.Empty:
                continue
            except Exception as e:
                print(f"❌ Error processing message in {self.name}: {e}")

        if self.debug_mode:
            print(f"🛑 {self.name} processing loop stopped")

# =============================================================================
# SCHEMA DEFINITIONS FOR COMMON AGENT CONFIGURATIONS
# =============================================================================

def create_agent_config_schema() -> ConfigSchema:
    """Create a comprehensive schema for agent configuration"""

    # Main schema
    schema = ConfigSchema("agent_system")

    # Global settings
    schema.add_field(ConfigField(
        name="debug_mode",
        config_type=ConfigurationType.BOOLEAN,
        default=False,
        description="Enable debug logging throughout the system",
        environment_variable="DEBUG_MODE"
    ))

    schema.add_field(ConfigField(
        name="environment",
        config_type=ConfigurationType.STRING,
        default="development",
        allowed_values=["development", "staging", "production"],
        description="Current deployment environment",
        environment_variable="ENVIRONMENT"
    ))

    # Agent defaults section
    agent_defaults_schema = ConfigSchema("agent_defaults")

    agent_defaults_schema.add_field(ConfigField(
        name="personality",
        config_type=ConfigurationType.STRING,
        default="balanced",
        allowed_values=["friendly", "professional", "casual", "analytical", "balanced"],
        description="Default personality for agents"
    ))

    agent_defaults_schema.add_field(ConfigField(
        name="max_response_length",
        config_type=ConfigurationType.INTEGER,
        default=200,
        min_value=50,
        max_value=1000,
        description="Maximum length of agent responses"
    ))

    agent_defaults_schema.add_field(ConfigField(
        name="processing_delay",
        config_type=ConfigurationType.FLOAT,
        default=0.01,
        min_value=0.0,
        max_value=1.0,
        description="Artificial delay for message processing (seconds)"
    ))

    schema.add_section("agents", ConfigSchema("agents"))
    schema.sections["agents"].add_section("default", agent_defaults_schema)

    # Individual agent configurations (will be added dynamically)

    # System configuration
    system_schema = ConfigSchema("system")

    system_schema.add_field(ConfigField(
        name="max_agents",
        config_type=ConfigurationType.INTEGER,
        default=100,
        min_value=1,
        max_value=1000,
        description="Maximum number of agents in the system"
    ))

    system_schema.add_field(ConfigField(
        name="message_queue_size",
        config_type=ConfigurationType.INTEGER,
        default=1000,
        min_value=10,
        max_value=10000,
        description="Maximum size of agent message queues"
    ))

    schema.add_section("system", system_schema)

    return schema

# =============================================================================
# INITIALIZATION COMPLETE
# =============================================================================

print("🔧 Tutorial 6 initialization complete!")
print("✅ All classes loaded successfully:")
print("   - ConfigField and ConfigSchema for validation")
print("   - ConfigManager for centralized configuration")
print("   - ConfigurableAgent with dynamic reconfiguration")
print("   - Schema definitions for agent systems")
print()
print("🚀 Ready to build flexible, configurable agent systems!")
print()

🔧 Tutorial 6 initialization complete!
✅ All classes loaded successfully:
   - ConfigField and ConfigSchema for validation
   - ConfigManager for centralized configuration
   - ConfigurableAgent with dynamic reconfiguration
   - Schema definitions for agent systems

🚀 Ready to build flexible, configurable agent systems!



In [6]:
# DEMO SECTION: Let's configure our agents dynamically!
# =============================================================================

print("=" * 62)
print("🚀 Tutorial 6: Configuration Management - Customizing Your Agents")
print("=" * 62)
print()


🚀 Tutorial 6: Configuration Management - Customizing Your Agents



In [7]:
# Step 1: Create configuration schema
print("📝 Step 1: Creating configuration schema...")
config_schema = create_agent_config_schema()

# Add specific agent configurations to schema
agent_names = ["alice", "bob", "charlie"]
for agent_name in agent_names:
    agent_schema = ConfigSchema(f"agent_{agent_name}")

    agent_schema.add_field(ConfigField(
        name="personality",
        config_type=ConfigurationType.STRING,
        allowed_values=["friendly", "professional", "casual", "analytical", "balanced"],
        description=f"Personality for {agent_name}"
    ))

    agent_schema.add_field(ConfigField(
        name="max_response_length",
        config_type=ConfigurationType.INTEGER,
        min_value=50,
        max_value=1000,
        description=f"Max response length for {agent_name}"
    ))

    agent_schema.add_field(ConfigField(
        name="debug_mode",
        config_type=ConfigurationType.BOOLEAN,
        description=f"Debug mode for {agent_name}"
    ))

    config_schema.sections["agents"].add_section(agent_name, agent_schema)

print("   ✅ Configuration schema created with validation rules")
print(f"   📋 Schema includes: global settings, agent defaults, and {len(agent_names)} specific agents")
print()

📝 Step 1: Creating configuration schema...
   ✅ Configuration schema created with validation rules
   📋 Schema includes: global settings, agent defaults, and 3 specific agents



In [8]:
# Step 2: Create ConfigManager and load base configuration
print("📝 Step 2: Creating ConfigManager and loading configuration...")

# Create config manager with schema
config_manager = ConfigManager(config_schema)
config_manager.set_environment("development")

# Create base configuration programmatically (simulating config file)
base_config = {
    "debug_mode": True,
    "environment": "development",
    "agents": {
        "default": {
            "personality": "balanced",
            "max_response_length": 150,
            "processing_delay": 0.02
        },
        "alice": {
            "personality": "friendly",
            "max_response_length": 200,
            "debug_mode": True
        },
        "bob": {
            "personality": "analytical",
            "max_response_length": 300,
            "processing_delay": 0.05
        },
        "charlie": {
            "personality": "professional",
            "debug_mode": False
        }
    },
    "system": {
        "max_agents": 50,
        "message_queue_size": 500
    }
}

config_manager.load_from_dict(base_config, "base_config")
config_manager.apply_defaults()

print("   ✅ Base configuration loaded")
print()


📝 Step 2: Creating ConfigManager and loading configuration...
🔧 ConfigManager initialized
   Schema: agent_system
🌍 Environment set to: development
📁 Loaded configuration from base_config
🔧 Applied default values for missing configuration fields
   ✅ Base configuration loaded



In [9]:
# Step 3: Validate configuration
print("📝 Step 3: Validating configuration...")
is_valid, errors = config_manager.validate()

if is_valid:
    print("   ✅ Configuration validation passed")
else:
    print("   ❌ Configuration validation failed:")
    for error in errors:
        print(f"      - {error}")
print()

📝 Step 3: Validating configuration...
   ✅ Configuration validation passed



In [10]:
# Step 4: Create configurable agents
print("📝 Step 4: Creating configurable agents...")

agents = []
for agent_name in agent_names:
    agent = ConfigurableAgent(
        name=agent_name.capitalize(),
        config_manager=config_manager
    )
    agents.append(agent)
    agent.start()

print(f"   ✅ Created and started {len(agents)} configurable agents")
print()


📝 Step 4: Creating configurable agents...
🤖 Initialized ConfigurableAgent: Alice
🔧 Alice configuration loaded:
   Personality: balanced
   Max response length: 150
   Processing delay: 0.02s
   Debug mode: True
🔔 Added configuration change listener
🔄 Alice processing loop started
▶️  ConfigurableAgent Alice started
🤖 Initialized ConfigurableAgent: Bob
🔔 Added configuration change listener
▶️  ConfigurableAgent Bob started
🤖 Initialized ConfigurableAgent: Charlie
🔔 Added configuration change listener
▶️  ConfigurableAgent Charlie started
   ✅ Created and started 3 configurable agents



In [11]:
# Step 5: Test initial configuration
print("📝 Step 5: Testing agents with initial configuration...")

test_messages = [
    "Hello there!",
    "What do you think about configuration management?",
    "Goodbye for now!"
]

for i, message_content in enumerate(test_messages, 1):
    print(f"--- Test Message {i}: '{message_content}' ---")

    for agent in agents:
        test_msg = Message.create(
            sender="config_tester",
            recipients=[agent.id],
            content=message_content
        )
        agent.receive_message(test_msg)

    time.sleep(0.5)
    print()


📝 Step 5: Testing agents with initial configuration...
--- Test Message 1: 'Hello there!' ---
📬 Alice received: Hello there!
🎯 Alice (balanced) processing: Hello there!
📤 Bob → config_t...: Hello there! It's wonderful to meet you! 😊
📤 Alice → config_t...: Hello there! It's wonderful to meet you! 😊
✅ Alice processed message in 0.020s
📤 Charlie → config_t...: Hello there! It's wonderful to meet you! 😊

--- Test Message 2: 'What do you think about configuration management?' ---
📬 Alice received: What do you think about configuration management?
🎯 Alice (balanced) processing: What do you think about configuration management?
📤 Alice → config_t...: Hello there! It's wonderful to meet you! 😊
✅ Alice processed message in 0.020s
📤 Charlie → config_t...: Hello there! It's wonderful to meet you! 😊
📤 Bob → config_t...: Hello there! It's wonderful to meet you! 😊

--- Test Message 3: 'Goodbye for now!' ---
📬 Alice received: Goodbye for now!
🎯 Alice (balanced) processing: Goodbye for now!
📤 Charlie 

In [12]:
# Step 6: Demonstrate dynamic configuration updates
print("📝 Step 6: Demonstrating dynamic configuration updates...")

print("   Changing Alice's personality from 'friendly' to 'casual'...")
config_manager.set("agents.alice.personality", "casual")

print("   Enabling debug mode globally...")
config_manager.set("debug_mode", True)

print("   Updating Bob's max response length...")
config_manager.set("agents.bob.max_response_length", 100)

time.sleep(0.5)  # Allow configuration changes to propagate

# Test with updated configuration
print("\n   Testing agents with updated configuration...")
test_msg = Message.create(
    sender="config_tester",
    recipients=[agent.id for agent in agents],
    content="Hey, how are you doing with the new config?"
)

for agent in agents:
    agent.receive_message(test_msg)

time.sleep(0.5)
print()

# Step 7: Environment-specific configuration
print("📝 Step 7: Testing environment-specific configuration...")

# Simulate production environment configuration
production_config = {
    "debug_mode": False,
    "environment": "production",
    "agents": {
        "default": {
            "processing_delay": 0.001,  # Faster in production
            "max_response_length": 100   # Shorter responses in prod
        },
        "alice": {
            "personality": "professional"  # More professional in production
        }
    },
    "system": {
        "max_agents": 200,
        "message_queue_size": 2000
    }
}

print("   Switching to production environment...")
config_manager.set_environment("production")
config_manager.load_from_dict(production_config, "production_config")

# Test production configuration
time.sleep(0.5)
print("\n   Testing agents in production configuration...")
prod_test_msg = Message.create(
    sender="prod_tester",
    recipients=[agents[0].id],  # Test with Alice
    content="Hello, this is a production test message!"
)

agents[0].receive_message(prod_test_msg)
time.sleep(0.5)
print()

📝 Step 6: Demonstrating dynamic configuration updates...
   Changing Alice's personality from 'friendly' to 'casual'...
🔧 Configuration updated: agents.alice.personality = casual (was: friendly)
🔔 Alice received configuration change notification
🔧 Alice configuration loaded:
   Personality: balanced
   Max response length: 150
   Processing delay: 0.02s
   Debug mode: True
   Enabling debug mode globally...
🔧 Configuration updated: debug_mode = True (was: True)
🔔 Alice received configuration change notification
🔧 Alice configuration loaded:
   Personality: balanced
   Max response length: 150
   Processing delay: 0.02s
   Debug mode: True
   Updating Bob's max response length...
🔧 Configuration updated: agents.bob.max_response_length = 100 (was: 300)
🔔 Alice received configuration change notification
🔧 Alice configuration loaded:
   Personality: balanced
   Max response length: 150
   Processing delay: 0.02s
   Debug mode: True

   Testing agents with updated configuration...
📬 Alice r

In [13]:
# Step 8: Configuration with environment variables (simulation)
print("📝 Step 8: Testing environment variable integration...")

# Simulate environment variables
test_env_vars = {
    "DEBUG_MODE": "false",
    "ENVIRONMENT": "staging"
}

# Temporarily set environment variables
original_env = {}
for key, value in test_env_vars.items():
    original_env[key] = os.environ.get(key)
    os.environ[key] = value

try:
    config_manager.load_from_environment()
    print("   ✅ Environment variables loaded")

    # Show current configuration
    print("   Current configuration status:")
    status = config_manager.get_status()
    for key, value in status.items():
        print(f"     {key}: {value}")

finally:
    # Restore original environment
    for key, original_value in original_env.items():
        if original_value is None:
            os.environ.pop(key, None)
        else:
            os.environ[key] = original_value

print()


📝 Step 8: Testing environment variable integration...
📁 Loaded configuration from environment
🔔 Alice received configuration change notification
🔧 Alice configuration loaded:
   Personality: professional
   Max response length: 100
   Processing delay: 0.001s
   Debug mode: True
🌍 Loaded 2 environment variables: ['DEBUG_MODE', 'ENVIRONMENT']
   ✅ Environment variables loaded
   Current configuration status:
     environment: production
     sources: ['base_config', 'production_config', 'environment']
     last_loaded: 1748131361.9018264
     config_keys: ['debug_mode', 'environment', 'agents', 'system']
     has_schema: True
     listeners: 3



In [14]:
# Step 9: Configuration file simulation and validation
print("📝 Step 9: Testing configuration validation and error handling...")

# Test invalid configuration
invalid_config = {
    "agents": {
        "alice": {
            "personality": "invalid_personality",  # Invalid value
            "max_response_length": 2000,  # Exceeds maximum
            "processing_delay": -1.0  # Below minimum
        }
    }
}

print("   Testing with invalid configuration...")
temp_manager = ConfigManager(config_schema)
temp_manager.load_from_dict(invalid_config, "invalid_test")

is_valid, errors = temp_manager.validate()
print(f"   Validation result: {'✅ Valid' if is_valid else '❌ Invalid'}")

if not is_valid:
    print("   Validation errors found:")
    for error in errors:
        print(f"     - {error}")

print()


📝 Step 9: Testing configuration validation and error handling...
   Testing with invalid configuration...
🔧 ConfigManager initialized
   Schema: agent_system
📁 Loaded configuration from invalid_test
   Validation result: ❌ Invalid
   Validation errors found:
     - [agent_alice] Field 'personality' must be one of ['friendly', 'professional', 'casual', 'analytical', 'balanced'], got invalid_personality
     - [agent_alice] Field 'max_response_length' must be <= 1000, got 2000



In [15]:
# Step 10: Visualize configuration management
print("📝 Step 10: Creating configuration visualization...")

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Prepare configuration data for visualization
    agent_personalities = []
    agent_response_lengths = []
    agent_delays = []
    agent_names_viz = []

    for agent in agents:
        agent_names_viz.append(agent.name)
        agent_personalities.append(agent.personality)
        agent_response_lengths.append(agent.max_response_length)
        agent_delays.append(agent.processing_delay * 1000)  # Convert to ms

    # Create configuration dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Agent Personalities',
            'Response Length Limits',
            'Processing Delays (ms)',
            'Configuration Timeline'
        ),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )

    # Agent personalities
    personality_colors = {
        'friendly': 'lightblue',
        'analytical': 'lightgreen',
        'professional': 'lightcoral',
        'casual': 'lightyellow',
        'balanced': 'lightpink'
    }
    colors = [personality_colors.get(p, 'lightgray') for p in agent_personalities]

    fig.add_trace(
        go.Bar(
            x=agent_names_viz,
            y=[1] * len(agent_names_viz),  # All bars same height
            name='Personality',
            marker_color=colors,
            text=agent_personalities,
            textposition='auto'
        ),
        row=1, col=1
    )

    # Response lengths
    fig.add_trace(
        go.Bar(
            x=agent_names_viz,
            y=agent_response_lengths,
            name='Max Response Length',
            marker_color='lightsteelblue',
            text=agent_response_lengths,
            textposition='auto'
        ),
        row=1, col=2
    )

    # Processing delays
    fig.add_trace(
        go.Bar(
            x=agent_names_viz,
            y=agent_delays,
            name='Processing Delay',
            marker_color='lightcoral',
            text=[f"{d:.0f}ms" for d in agent_delays],
            textposition='auto'
        ),
        row=2, col=1
    )

    # Configuration timeline (simulated)
    config_events = ['Initial Load', 'Alice Update', 'Global Debug', 'Bob Update', 'Prod Switch']
    event_times = [i for i in range(len(config_events))]

    fig.add_trace(
        go.Scatter(
            x=event_times,
            y=[1, 2, 3, 2, 4],  # Simulated impact levels
            mode='lines+markers',
            name='Config Changes',
            text=config_events,
            line=dict(color='green', width=3),
            marker=dict(size=10)
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        title_text="Configuration Management Dashboard",
        height=800,
        showlegend=True,
        template='plotly_white'
    )

    # Update axes
    fig.update_xaxes(title_text="Agents", row=1, col=1)
    fig.update_yaxes(title_text="Personality Type", row=1, col=1)

    fig.update_xaxes(title_text="Agents", row=1, col=2)
    fig.update_yaxes(title_text="Characters", row=1, col=2)

    fig.update_xaxes(title_text="Agents", row=2, col=1)
    fig.update_yaxes(title_text="Milliseconds", row=2, col=1)

    fig.update_xaxes(title_text="Timeline", row=2, col=2)
    fig.update_yaxes(title_text="Impact Level", row=2, col=2)

    fig.show()

    print("   ✅ Configuration dashboard created!")
    print("   📊 The dashboard shows:")
    print("      - Agent personality distribution and assignments")
    print("      - Response length limits for each agent")
    print("      - Processing delay configurations")
    print("      - Timeline of configuration changes")
    print()

    # Create configuration source breakdown
    fig2 = go.Figure(data=go.Pie(
        labels=config_manager.config_sources,
        values=[1] * len(config_manager.config_sources),  # Equal weight for demo
        title="Configuration Sources"
    ))

    fig2.update_layout(
        title_text="Configuration Sources and Loading Order",
        template='plotly_white'
    )

    fig2.show()
    print("   ✅ Configuration sources visualization created!")
    print()

except ImportError:
    print("   ⚠️  Plotly not available - skipping visualization")
    print("   💡 To see configuration visualizations, install plotly: pip install plotly")
    print("   📊 Configuration summary:")
    print(f"      Environment: {config_manager.environment}")
    print(f"      Sources: {config_manager.config_sources}")
    print(f"      Active agents: {len(agents)}")
    print("      Agent configurations:")
    for agent in agents:
        print(f"        {agent.name}: {agent.personality}, max_len={agent.max_response_length}")
    print()


📝 Step 10: Creating configuration visualization...


   ✅ Configuration dashboard created!
   📊 The dashboard shows:
      - Agent personality distribution and assignments
      - Response length limits for each agent
      - Processing delay configurations
      - Timeline of configuration changes



   ✅ Configuration sources visualization created!



In [16]:
# Step 11: Advanced configuration patterns
print("📝 Step 11: Demonstrating advanced configuration patterns...")

# Configuration templates and inheritance
print("   Testing configuration inheritance...")
config_manager.set("agents.default.personality", "analytical")  # Change default
print("   → Changed default personality to 'analytical'")

# This should affect agents that don't have explicit personality settings
config_manager.reload()  # Reload to apply inheritance

print("   → Configuration inheritance applied")

# Configuration secrets handling
print("\n   Testing secure configuration handling...")
config_manager.set("database.password", "super_secret_password")
masked_password = "*" * len(config_manager.get("database.password", ""))
print(f"   → Database password configured: {masked_password}")

# Conditional configuration
environment = config_manager.get("environment")
if environment == "production":
    print(f"   → Production environment detected - applying security defaults")
    config_manager.set("debug_mode", False)
    config_manager.set("agents.default.processing_delay", 0.001)
else:
    print(f"   → {environment} environment - keeping development settings")

print()

# Step 12: Performance and monitoring
print("📝 Step 12: Configuration performance analysis...")

# Measure configuration access performance
import time

start_time = time.time()
for _ in range(1000):
    config_manager.get("agents.alice.personality")
access_time = (time.time() - start_time) * 1000

print(f"   Configuration access performance: {access_time:.2f}ms for 1000 operations")
print(f"   Average per access: {access_time/1000:.4f}ms")

# Configuration change propagation test
print("   Testing configuration change propagation...")
change_count = 0

def count_changes(cm):
    global change_count
    change_count += 1

config_manager.add_change_listener(count_changes)

# Make several changes
config_manager.set("test.value1", "test")
config_manager.set("test.value2", 42)
config_manager.set("test.value3", True)

print(f"   Configuration changes propagated: {change_count} events")
print()

# Step 13: Final system state and cleanup
print("📝 Step 13: Final system analysis and cleanup...")

# Get final configuration state
final_config = config_manager.get_all()
print("   Final system configuration summary:")
print(f"     Environment: {config_manager.environment}")
print(f"     Debug mode: {config_manager.get('debug_mode')}")
print(f"     Total configuration keys: {len(final_config)}")

# Agent final states
print("   Final agent configurations:")
for agent in agents:
    print(f"     {agent.name}:")
    print(f"       Personality: {agent.personality}")
    print(f"       Max response: {agent.max_response_length} chars")
    print(f"       Processing delay: {agent.processing_delay:.3f}s")
    print(f"       Debug mode: {agent.debug_mode}")

# Configuration validation report
is_valid, errors = config_manager.validate()
print(f"\n   Final configuration validation: {'✅ Valid' if is_valid else '❌ Invalid'}")
if errors:
    for error in errors:
        print(f"     ⚠️  {error}")

print()

# Stop all agents
print("   Stopping all agents...")
for agent in agents:
    agent.stop()

print("✅ Tutorial 6 Complete!")
print()


📝 Step 11: Demonstrating advanced configuration patterns...
   Testing configuration inheritance...
🔧 Configuration updated: agents.default.personality = analytical (was: None)
🔔 Alice received configuration change notification
🔧 Alice configuration loaded:
   Personality: analytical
   Max response length: 100
   Processing delay: 0.001s
   Debug mode: True
🎭 Alice personality changed: professional → analytical
🎭 Bob personality changed: balanced → analytical
🎭 Charlie personality changed: balanced → analytical
   → Changed default personality to 'analytical'
🔄 Reloading configuration...
📁 Loaded configuration from environment
🔔 Alice received configuration change notification
🔧 Alice configuration loaded:
   Personality: analytical
   Max response length: 100
   Processing delay: 0.001s
   Debug mode: True
🌍 Loaded 0 environment variables: []
🔧 Applied default values for missing configuration fields
✅ Configuration reloaded
   → Configuration inheritance applied

   Testing secure co

In [17]:
# SUMMARY OF WHAT WE LEARNED
# =============================================================================

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print("1. 🔧 Built a comprehensive configuration management system")

print("   - Schema-based configuration with validation")

print("   - Environment-specific configuration loading")

print("   - Dynamic configuration updates without restarts")

print("   - Environment variable integration")

print()

print("2. 📋 Implemented production-ready configuration patterns")

print("   - Configuration inheritance and defaults")

print("   - Secure handling of sensitive configuration")

print("   - Multi-source configuration merging")

print("   - Runtime configuration change propagation")

print()

print("3. 🎯 Created highly configurable agents")

print("   - Dynamic personality and behavior changes")

print("   - Configuration-driven processing parameters")

print("   - Automatic configuration reload capabilities")

print("   - Debug mode and development features")

print()

print("4. 📊 Added configuration monitoring and visualization")

print("   - Configuration change tracking and history")

print("   - Performance monitoring for config access")

print("   - Visual configuration dashboards")

print("   - Validation reporting and error handling")

print()

📚 WHAT WE LEARNED:
1. 🔧 Built a comprehensive configuration management system
   - Schema-based configuration with validation
   - Environment-specific configuration loading
   - Dynamic configuration updates without restarts
   - Environment variable integration

2. 📋 Implemented production-ready configuration patterns
   - Configuration inheritance and defaults
   - Secure handling of sensitive configuration
   - Multi-source configuration merging
   - Runtime configuration change propagation

3. 🎯 Created highly configurable agents
   - Dynamic personality and behavior changes
   - Configuration-driven processing parameters
   - Automatic configuration reload capabilities
   - Debug mode and development features

4. 📊 Added configuration monitoring and visualization
   - Configuration change tracking and history
   - Performance monitoring for config access
   - Visual configuration dashboards
   - Validation reporting and error handling



In [18]:
# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")

print("=" * 40)

print("1. 🐛 Configuration validation failures")

print("   Problem: Invalid values or missing required fields")

print("   Solution: Use schema validation before applying config")

print("   Solution: Provide meaningful error messages with field context")

print("   Solution: Apply defaults before validation")

print()

print("2. 🐛 Environment variable parsing errors")

print("   Problem: String env vars not converted to proper types")

print("   Solution: Define config_type in ConfigField for auto-conversion")

print("   Solution: Handle JSON parsing errors gracefully")

print("   Solution: Validate converted values against schema")

print()

print("3. 🐛 Configuration changes not propagating")

print("   Problem: Agents not receiving config change notifications")

print("   Solution: Ensure agents register as change listeners")

print("   Solution: Use thread-safe configuration updates")

print("   Solution: Test configuration callbacks separately")

print()

print("4. 🐛 Circular configuration dependencies")

print("   Problem: Config A depends on Config B which depends on Config A")

print("   Solution: Design flat configuration structures when possible")

print("   Solution: Use dependency ordering in configuration loading")

print("   Solution: Validate for circular references")

print()

print("5. 🐛 Performance issues with frequent config access")

print("   Problem: Configuration lookups causing bottlenecks")

print("   Solution: Cache frequently accessed configuration values")

print("   Solution: Use local copies for performance-critical code")

print("   Solution: Batch configuration updates when possible")

print()

print("6. 🐛 Configuration file format errors")

print("   Problem: YAML/JSON syntax errors or schema mismatches")

print("   Solution: Validate file format before loading")

print("   Solution: Provide clear error messages with line numbers")

print("   Solution: Use configuration file templates")

print()

print("🎉 Ready for Tutorial 7: CLI Basics!")

print("   Next we'll add command-line interfaces for agent management...")

⚠️  COMMON ERRORS AND SOLUTIONS:
1. 🐛 Configuration validation failures
   Problem: Invalid values or missing required fields
   Solution: Use schema validation before applying config
   Solution: Provide meaningful error messages with field context
   Solution: Apply defaults before validation

2. 🐛 Environment variable parsing errors
   Problem: String env vars not converted to proper types
   Solution: Define config_type in ConfigField for auto-conversion
   Solution: Handle JSON parsing errors gracefully
   Solution: Validate converted values against schema

3. 🐛 Configuration changes not propagating
   Problem: Agents not receiving config change notifications
   Solution: Ensure agents register as change listeners
   Solution: Use thread-safe configuration updates
   Solution: Test configuration callbacks separately

4. 🐛 Circular configuration dependencies
   Problem: Config A depends on Config B which depends on Config A
   Solution: Design flat configuration structures when pos